In [10]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import StandardScaler




In [11]:
# Load datasets
customers = pd.read_csv('Customers.csv')
products = pd.read_csv('Products.csv')
transactions = pd.read_csv('Transactions.csv')

In [12]:
# Merge datasets to create a comprehensive customer profile

transactions = transactions.merge(products, on='ProductID', suffixes=('_transaction', '_product'))
transactions = transactions.merge(customers, on='CustomerID', suffixes=('', '_customer'))



In [4]:
transactions.columns

Index(['TransactionID', 'CustomerID', 'ProductID', 'TransactionDate',
       'Quantity', 'TotalValue', 'Price_transaction', 'ProductName',
       'Category', 'Price_product', 'CustomerName', 'Region', 'SignupDate'],
      dtype='object')

In [13]:
# Select relevant columns and remove duplicates
transactions = transactions[[
    'TransactionID', 'CustomerID', 'ProductID', 'TransactionDate', 
    'Quantity', 'TotalValue', 'Price_product', 'ProductName', 'Category',
    'CustomerName', 'Region', 'SignupDate'
]]

# Rename columns for clarity
transactions.rename(columns={
    'Price_product': 'ProductPrice',
    'ProductName': 'ProductName',
    'Category': 'ProductCategory',
    'CustomerName': 'CustomerName',
    'Region': 'CustomerRegion',
    'SignupDate': 'CustomerSignupDate'
}, inplace=True)

# Display the cleaned DataFrame
print(transactions.head())


  TransactionID CustomerID ProductID      TransactionDate  Quantity  \
0        T00001      C0199      P067  2024-08-25 12:38:23         1   
1        T00112      C0146      P067  2024-05-27 22:23:54         1   
2        T00166      C0127      P067  2024-04-25 07:38:55         1   
3        T00272      C0087      P067  2024-03-26 22:55:37         2   
4        T00363      C0070      P067  2024-03-21 15:10:10         3   

   TotalValue  ProductPrice                      ProductName ProductCategory  \
0      300.68        300.68  ComfortLiving Bluetooth Speaker     Electronics   
1      300.68        300.68  ComfortLiving Bluetooth Speaker     Electronics   
2      300.68        300.68  ComfortLiving Bluetooth Speaker     Electronics   
3      601.36        300.68  ComfortLiving Bluetooth Speaker     Electronics   
4      902.04        300.68  ComfortLiving Bluetooth Speaker     Electronics   

      CustomerName CustomerRegion CustomerSignupDate  
0   Andrea Jenkins         Europe    

In [14]:


# Step 1: Feature Engineering
# Create features for each customer
customer_profile = transactions.groupby('CustomerID').agg({
    'TotalValue': 'sum',  # Total spending
    'TransactionID': 'count',  # Number of purchases
    'Quantity': 'sum',  # Total quantity purchased
    'ProductPrice': 'mean',  # Average product price
    'ProductID': lambda x: x.mode()[0]  # Most purchased product
}).reset_index()



In [15]:
customer_profile.rename(columns={
    'TotalValue': 'TotalSpending',
    'TransactionID': 'TotalTransactions',
    'Quantity': 'TotalQuantity',
    'ProductPrice': 'AvgProductPrice',
    'ProductID': 'FavoriteProduct'
}, inplace=True)

# Convert categorical 'FavoriteProduct' into numerical features using one-hot encoding
customer_profile = pd.get_dummies(customer_profile, columns=['FavoriteProduct'], drop_first=True)

# Merge demographic data from Customers.csv
customer_profile = customer_profile.merge(customers, on='CustomerID')

# Step 2: Normalize Features for Similarity Computation
scaler = StandardScaler()
scaled_features = scaler.fit_transform(customer_profile.drop(columns=['CustomerID', 'CustomerName', 'Region', 'SignupDate']))

# Step 3: Compute Similarity Matrix
similarity_matrix = cosine_similarity(scaled_features)

# Step 4: Find Top 3 Lookalikes for Each Customer
lookalike_map = {}
for idx, customer_id in enumerate(customer_profile['CustomerID']):
    # Get similarity scores for the customer
    sim_scores = list(enumerate(similarity_matrix[idx]))
    # Sort by similarity score in descending order and exclude the customer themselves
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)[1:]
    # Get the top 3 similar customers
    top_3 = [(customer_profile.iloc[i]['CustomerID'], score) for i, score in sim_scores[:3]]
    lookalike_map[customer_id] = top_3

# Step 5: Generate Lookalike.csv
lookalike_data = []
for cust_id, lookalikes in lookalike_map.items():
    lookalike_data.append({
        'cust_id': cust_id,
        'lookalikes': lookalikes
    })

lookalike_df = pd.DataFrame(lookalike_data)
lookalike_df.to_csv('Sapna_Saini_Lookalike.csv', index=False)

# Step 6: Filter for the First 20 Customers
filtered_lookalike_df = lookalike_df[lookalike_df['cust_id'].isin(customers['CustomerID'][:20])]
print(filtered_lookalike_df)


   cust_id                                         lookalikes
0    C0001  [(C0179, 0.9916346221269137), (C0160, 0.989230...
1    C0002  [(C0030, 0.9924792466812008), (C0164, 0.980387...
2    C0003  [(C0031, 0.987097662098253), (C0181, 0.9712021...
3    C0004  [(C0175, 0.9837947292638719), (C0057, 0.960635...
4    C0005  [(C0149, 0.9687578272292461), (C0023, 0.940558...
5    C0006  [(C0040, 0.9888769559538623), (C0114, 0.981958...
6    C0007  [(C0009, 0.985597868137347), (C0112, 0.9672417...
7    C0008  [(C0173, 0.9761371057144449), (C0030, 0.917658...
8    C0009  [(C0007, 0.985597868137347), (C0112, 0.9798007...
9    C0010  [(C0034, 0.9883684199056102), (C0071, 0.156432...
10   C0011  [(C0171, 0.9913508868399603), (C0073, 0.977605...
11   C0012  [(C0133, 0.9598859142662605), (C0128, 0.872283...
12   C0013  [(C0021, 0.9973904838876342), (C0101, 0.993665...
13   C0014  [(C0071, 0.2127296481419964), (C0036, 0.201713...
14   C0015  [(C0126, 0.9536137115590314), (C0036, 0.136864...
15   C00

In [17]:
# print(customers[customers['CustomerID'] == 'C0180'])


    CustomerID   CustomerName Region  SignupDate
179      C0180  Amy Carpenter   Asia  2023-10-25


In [18]:
# print(transactions[transactions['CustomerID'] == 'C0180'])


Empty DataFrame
Columns: [TransactionID, CustomerID, ProductID, TransactionDate, Quantity, TotalValue, ProductPrice, ProductName, ProductCategory, CustomerName, CustomerRegion, CustomerSignupDate]
Index: []
